In [49]:
import sys
import os
import boto3
import json

import xml.etree.ElementTree as ET

sys.path.insert(0, "../../src")
import GOSTrocks.dataMisc as dMisc

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
s3_bucket = "wbg-geography01"
s3_prefix = "FATHOM/v2023/"
s3_out = os.path.join("s3://", s3_bucket, s3_prefix)

s3 = boto3.resource("s3")
my_bucket = s3.Bucket(s3_bucket)

# Find all files already copied
all_folders = []
for obj in my_bucket.objects.filter(Prefix=s3_prefix):
    all_folders.append(obj.key.split("/")[-2])
all_folders
all_folders = list(set(all_folders))

In [ ]:
# build list of rasters for generating VRT
local_path = os.path.join(
    "v2023",
    "GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0",
)
in_folder = os.path.join(template_folder, local_path)
all_tiffs = [f"{local_path}/{x}" for x in os.listdir(in_folder)]

with open(os.path.join(template_folder, "s3_tiffs.txt"), "w") as out:
    for p in all_tiffs:
        out.write(f"{p}\n")

In [ ]:
template_folder = "/home/wb411133/temp"
coastal_template = os.path.join(
    template_folder,
    "GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0.0.vrt",
)
other_template = os.path.join(
    template_folder,
    "GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0.vrt",
)


class generate_vrt_from_template:
    def __init__(self, template_vrt, new_dataset):
        self.template_vrt = template_vrt
        self.new_dataset = new_dataset
        self.old_dataset = os.path.basename(template_vrt)[:-4]
        self.new_vrt = self.template_vrt.replace(self.old_dataset, self.new_dataset)

    def update_vrt(self):
        tree = ET.parse(self.template_vrt)
        root = tree.getroot()

        for child in root.iter("SourceFilename"):
            child.text = child.text.replace(self.old_dataset, self.new_dataset)

        tree.write(self.new_vrt, xml_declaration=False)


"""
new_ds = "GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP3_7.0-PERCENTILE50-v3.0"
xx = generate_vrt_from_template(template_vrt, new_ds)
xx.update_vrt()
"""

In [ ]:
# Create template vrts
for new_ds in all_folders:
    flood_type = new_ds.split("-")[4]
    vrt_template = other_template
    if flood_type == "COASTAL":
        vrt_template = coastal_template
    xx = generate_vrt_from_template(vrt_template, new_ds)
    xx.update_vrt()

In [ ]:
# aws s3 cp . s3://wbg-geography01/FATHOM/ --exclude "*" --include "*.vrt"  --recursive

# Inspect copied VRTs

In [ ]:
dMisc.get_fathom_vrts()

In [ ]:
s3_bucket = "wbg-geography01"
s3_prefix = "FATHOM/"

s3 = boto3.resource("s3")
my_bucket = s3.Bucket(s3_bucket)

all_vrts = []
for o in my_bucket.objects.filter(Prefix=s3_prefix):
    if o.key.endswith(".vrt"):
        print(o.key)
        full_vrt_path = f"s3://{s3_bucket}/{o.key}"
        all_vrts.append(full_vrt_path)

In [53]:
all_vrts.head()

,RETURN,FLOOD_TYPE,DEFENCE,DEPTH,YEAR,CLIMATE_MODEL,PATH
0,1in10,COASTAL,DEFENDED,DEPTH,2020,PERCENTILE50,s3://wbg-geography01/FATHOM/GLOBAL-1ARCSEC-NW_...
1,1in10,COASTAL,DEFENDED,DEPTH,2030,SSP1_2.6,s3://wbg-geography01/FATHOM/GLOBAL-1ARCSEC-NW_...
2,1in10,COASTAL,DEFENDED,DEPTH,2030,SSP2_4.5,s3://wbg-geography01/FATHOM/GLOBAL-1ARCSEC-NW_...
3,1in10,COASTAL,DEFENDED,DEPTH,2030,SSP3_7.0,s3://wbg-geography01/FATHOM/GLOBAL-1ARCSEC-NW_...
4,1in10,COASTAL,DEFENDED,DEPTH,2030,SSP5_8.5,s3://wbg-geography01/FATHOM/GLOBAL-1ARCSEC-NW_...


In [55]:
all_vrts["FLOOD_TYPE"].value_counts()

COASTAL    208
FLUVIAL    208
PLUVIAL    104
Name: FLOOD_TYPE, dtype: int64

In [47]:
all_vrts = dMisc.get_fathom_vrts(True)
all_res = {}
for idx, row in all_vrts.iterrows():
    # vrt_path = row['PATH']
    # xx = rasterio.open(vrt_path)
    filename = os.path.basename(row["PATH"])
    year = row["YEAR"]
    climate_model = row["CLIMATE_MODEL"]
    if climate_model == "PERCENTILE50":
        climate_model = "CURRENT"
    flood_type = row["FLOOD_TYPE"].lower()
    defence = row["DEFENCE"].lower()
    ret = row["RETURN"]
    label = "_".join([flood_type, defence, ret, climate_model, year])
    ret = ret.replace("in", " in ")
    if year == "2020":
        description = f"Global {defence} {flood_type} flood model based on current climate. Flood depth is measured in cm expected flood depth, based on a {ret} year return period."
    else:
        description = f"Global {defence} {flood_type} flood model based on {climate_model} climate model for year {year}. Flood depth is measured in cm expected flood depth, based on a {ret} year return period."
    all_res[label] = {"description": description, "filename": row["PATH"]}

In [48]:
all_res

{'coastal_defended_1in10_CURRENT_2020': {'description': 'Global defended coastal flood model based on current climate. Flood depth is measured in cm expected flood depth, based on a 1 in 10 year return period.',
  'filename': 's3://wbg-geography01/FATHOM/GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0.vrt'},
 'coastal_defended_1in10_SSP1_2.6_2030': {'description': 'Global defended coastal flood model based on SSP1_2.6 climate model for year 2030. Flood depth is measured in cm expected flood depth, based on a 1 in 10 year return period.',
  'filename': 's3://wbg-geography01/FATHOM/GLOBAL-1ARCSEC-NW_OFFSET-1in10-COASTAL-DEFENDED-DEPTH-2030-SSP1_2.6-PERCENTILE50-v3.0.vrt'},
 'coastal_defended_1in10_SSP2_4.5_2030': {'description': 'Global defended coastal flood model based on SSP2_4.5 climate model for year 2030. Flood depth is measured in cm expected flood depth, based on a 1 in 10 year return period.',
  'filename': 's3://wbg-geography01/FATHOM/GLOBAL-1ARCSEC

In [52]:
with open("fathom_file_descriptions.json", "w") as out_f:
    json.dump(all_res, out_f)

In [51]:
json.dump?

# DEBURGGING

In [ ]:
# Generate a list of files for gdalbuildvrt
folder = "/home/wb411133/temp/v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0"
all_files = [
    f"v2023/GLOBAL-1ARCSEC-NW_OFFSET-1in500-PLUVIAL-DEFENDED-DEPTH-2020-PERCENTILE50-v3.0/{x}"
    for x in os.listdir(folder)
]
all_files